In [107]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plot
import itertools as it
import sklearn.linear_model as lm
from sklearn.metrics import mean_squared_error, r2_score

names = ["Tom","Jackson","Ishjot","David","Ruvan","Austin","Theo","Josh","Vinny*","Andrew","Oliver", "Michael"]

data = np.loadtxt(open("AFAPy.csv"), delimiter=",",skiprows=1)
datat = np.transpose(data)
logit = lm.LogisticRegression(solver="sag",multi_class="multinomial")
linear = lm.LinearRegression()
TTKReg = lm.LinearRegression()
dubs = datat[16]

players = data[0:,1:13]

In [108]:
logit.fit(players, dubs)
print("Multinomial Logistic Regression without Interactions: Predicting Dub Rate by player only")
print("Intercept: \n", logit.intercept_)
print("Coefficients: \n", logit.coef_)
logitInt = logit.intercept_
logitCoefs = logit.coef_
predictions = logit.predict(players)
print("MSE: ", mean_squared_error(dubs, predictions))
print('R^2: %.2f' % r2_score(dubs, predictions))

Multinomial Logistic Regression without Interactions: Predicting Dub Rate by player only
Intercept: 
 [-1.99743207]
Coefficients: 
 [[ 0.55197788  0.19035595  0.45597952  0.09292183  0.32022468  0.32251722
   0.37382396  0.30475372  0.64381871 -0.03194677 -0.13618801  0.64330323]]
MSE:  0.193406593407
R^2: -0.24


In [109]:
adjTTK = datat[15]
linear.fit(players, adjTTK)
print("General Linear Regression without Interactions: Predicting AdjTTK by players only")
print("Intercept: \n", linear.intercept_)
print("Coefficients: \n", linear.coef_)
TTKInt = linear.intercept_
TTKCoefs = linear.coef_
predictions = linear.predict(players)
print("MSE: ", mean_squared_error(adjTTK, predictions))
print('R^2: %.2f' % linear.score(players, adjTTK))

General Linear Regression without Interactions: Predicting AdjTTK by players only
Intercept: 
 6.71029211961
Coefficients: 
 [ 2.15460147  0.71155481  0.37247    -1.01061056 -1.38943496 -0.60577056
  0.91722983  1.6065247   1.57621461 -1.79465145  1.35842367  7.32987438]
MSE:  36.9763958971
R^2: 0.05


In [110]:
AP = datat[13]
TTKReg.fit(players, AP)
print("General Linear Regression without Interactions: Predicting AP by players only")
print("Intercept: \n", TTKReg.intercept_)
print("Coefficients: \n", TTKReg.coef_)
APInt = TTKReg.intercept_
APCoefs = TTKReg.coef_
predictions = TTKReg.predict(players)
print("MSE: ", mean_squared_error(AP, predictions))
print('R^2: %.2f' % TTKReg.score(players, AP))

General Linear Regression without Interactions: Predicting AP by players only
Intercept: 
 20.4699204413
Coefficients: 
 [ -4.31246692e+00  -2.52624836e+00  -3.41971740e+00  -1.59612902e+00
  -1.39693204e+00  -1.56066534e+00  -3.37349973e+00  -2.57185093e+00
  -3.76867241e+00   6.12723126e-04  -5.97025072e+00  -1.25444537e+01]
MSE:  73.1680819776
R^2: 0.04


In [111]:
print("Name \t\tMultiplicative Effect on Dub Odds Ratio \t Effect on teams AdjTTK \t Effect on AP")
for i in range(0, len(names)):
    print(names[i], "\t\t", round(np.exp(logitCoefs[0][i]),3),"\t \t \t\t\t\t",round(TTKCoefs[i],3),"\t\t\t\t",round(APCoefs[i],3))
print("Coefficents \t",round(np.exp(logitInt[0]),3),"\t\t\t\t\t\t",round(TTKInt,3),"\t\t\t\t",round(APInt,3))

Name 		Multiplicative Effect on Dub Odds Ratio 	 Effect on teams AdjTTK 	 Effect on AP
Tom 		 1.737 	 	 				 2.155 				 -4.312
Jackson 		 1.21 	 	 				 0.712 				 -2.526
Ishjot 		 1.578 	 	 				 0.372 				 -3.42
David 		 1.097 	 	 				 -1.011 				 -1.596
Ruvan 		 1.377 	 	 				 -1.389 				 -1.397
Austin 		 1.381 	 	 				 -0.606 				 -1.561
Theo 		 1.453 	 	 				 0.917 				 -3.373
Josh 		 1.356 	 	 				 1.607 				 -2.572
Vinny* 		 1.904 	 	 				 1.576 				 -3.769
Andrew 		 0.969 	 	 				 -1.795 				 0.001
Oliver 		 0.873 	 	 				 1.358 				 -5.97
Michael 		 1.903 	 	 				 7.33 				 -12.544
Coefficents 	 0.136 						 6.71 				 20.47


In [112]:
combos = set(it.combinations(names, 4))
odds = []
probs = []
teamOdds = {}
teamProbs = {}
for team in combos:
    odd = logitInt[0]
    for player in team:
        odd = odd + logitCoefs[0][names.index(player)]
    odds.append(odd)
    probs.append(np.exp(odd)/(1+np.exp(odd)))
    teamOdds[team] = odd
    teamProbs[team] = np.exp(odd)/(1+np.exp(odd))
teamProbs
sorted(teamProbs.items(), key=lambda x: x[1])

[(('Jackson', 'David', 'Andrew', 'Oliver'), 0.13212616669614305),
 (('David', 'Josh', 'Andrew', 'Oliver'), 0.14580476124985495),
 (('David', 'Ruvan', 'Andrew', 'Oliver'), 0.1477421816329528),
 (('David', 'Austin', 'Andrew', 'Oliver'), 0.14803107895094611),
 (('David', 'Theo', 'Andrew', 'Oliver'), 0.15461930354987122),
 (('Jackson', 'Josh', 'Andrew', 'Oliver'), 0.15836323363034985),
 (('Jackson', 'Ruvan', 'Andrew', 'Oliver'), 0.1604361855900345),
 (('Jackson', 'Austin', 'Andrew', 'Oliver'), 0.16074522324499121),
 (('Ishjot', 'David', 'Andrew', 'Oliver'), 0.1656652505531763),
 (('Jackson', 'Theo', 'Andrew', 'Oliver'), 0.16778785998188098),
 (('Jackson', 'David', 'Josh', 'Oliver'), 0.17572432711142316),
 (('Ruvan', 'Josh', 'Andrew', 'Oliver'), 0.17644974694918342),
 (('Austin', 'Josh', 'Andrew', 'Oliver'), 0.17678313539818366),
 (('Jackson', 'David', 'Ruvan', 'Oliver'), 0.17797647704085456),
 (('Jackson', 'David', 'Austin', 'Oliver'), 0.17831212555881931),
 (('Ruvan', 'Austin', 'Andrew', 

In [113]:
combos = set(it.combinations(names, 4))
TTKs = []
teamTTKs = {}
for team in combos:
    kills = TTKInt
    for player in team:
        kills = kills + TTKCoefs[names.index(player)]
    TTKs.append(kills)
    teamTTKs[team] = kills
sorted(teamTTKs.items(), key=lambda x: x[1])

[(('David', 'Ruvan', 'Austin', 'Andrew'), 1.9098245974557573),
 (('Ishjot', 'David', 'Ruvan', 'Andrew'), 2.8880651567282527),
 (('Jackson', 'David', 'Ruvan', 'Andrew'), 3.227149970004958),
 (('Ishjot', 'Ruvan', 'Austin', 'Andrew'), 3.2929051540657213),
 (('David', 'Ruvan', 'Theo', 'Andrew'), 3.4328249920333231),
 (('Jackson', 'Ruvan', 'Austin', 'Andrew'), 3.6319899673424265),
 (('Ishjot', 'David', 'Austin', 'Andrew'), 3.6717295525482871),
 (('Ruvan', 'Austin', 'Theo', 'Andrew'), 3.8376649893707917),
 (('David', 'Ruvan', 'Andrew', 'Oliver'), 3.8740188278058207),
 (('Jackson', 'David', 'Austin', 'Andrew'), 4.0108143658249924),
 (('Ishjot', 'David', 'Ruvan', 'Austin'), 4.0769460433626774),
 (('David', 'Ruvan', 'Vinny*', 'Andrew'), 4.0918097724179265),
 (('David', 'Ruvan', 'Josh', 'Andrew'), 4.1221198582643801),
 (('David', 'Austin', 'Theo', 'Andrew'), 4.2164893878533576),
 (('Ruvan', 'Austin', 'Andrew', 'Oliver'), 4.2788588251432893),
 (('Jackson', 'David', 'Ruvan', 'Austin'), 4.416030856

In [114]:
combos = set(it.combinations(names, 4))
APs = []
TeamAPs = {}
for team in combos:
    Place = APInt
    for player in team:
        Place = Place + APCoefs[names.index(player)]
    APs.append(Place)
    TeamAPs[team] = Place
sorted(TeamAPs.items(), key=lambda x: x[1])

[(('Tom', 'Vinny*', 'Oliver', 'Michael'), -6.1259232675072086),
 (('Tom', 'Ishjot', 'Oliver', 'Michael'), -5.7769682618320717),
 (('Tom', 'Theo', 'Oliver', 'Michael'), -5.7307505881906105),
 (('Ishjot', 'Vinny*', 'Oliver', 'Michael'), -5.2331737560820608),
 (('Theo', 'Vinny*', 'Oliver', 'Michael'), -5.1869560824405996),
 (('Tom', 'Josh', 'Oliver', 'Michael'), -4.9291017879288797),
 (('Tom', 'Jackson', 'Oliver', 'Michael'), -4.8834992171744824),
 (('Ishjot', 'Theo', 'Oliver', 'Michael'), -4.8380010767654626),
 (('Josh', 'Vinny*', 'Oliver', 'Michael'), -4.3853072821788679),
 (('Jackson', 'Vinny*', 'Oliver', 'Michael'), -4.3397047114244707),
 (('Ishjot', 'Josh', 'Oliver', 'Michael'), -4.036352276503731),
 (('Jackson', 'Ishjot', 'Oliver', 'Michael'), -3.9907497057493337),
 (('Theo', 'Josh', 'Oliver', 'Michael'), -3.9901346028622697),
 (('Tom', 'David', 'Oliver', 'Michael'), -3.9533798811357137),
 (('Jackson', 'Theo', 'Oliver', 'Michael'), -3.9445320321078725),
 (('Tom', 'Austin', 'Oliver',